<a href="https://colab.research.google.com/github/rgumi/dataScience/blob/master/seminarpaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, make_scorer
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from scipy.stats import randint

In [12]:
data = pd.read_csv('https://raw.githubusercontent.com/saschaschworm/big-data-and-data-science/master/datasets/prediction-challenge/dataset.csv')
X, y = data.iloc[:, 1:-1], data['success']

hyperparams = {'random_state': 1909}
model = RandomForestClassifier(**hyperparams)
data.head()

,identifier,date,age,marital_status,education,job,credit_default,housing_loan,personal_loan,communication_type,n_contacts_campaign,days_since_last_contact,n_contacts_before,previous_conversion,success
0,34203,2009-05-06,36,divorced,High School,Service provider,No,No,Yes,Landline network,5,-1,0,Inexistent,No
1,1250,2008-05-08,32,single,University,Student,No,Yes,No,Landline network,1,-1,0,Inexistent,No
2,38130,2009-09-04,68,single,University,Pensioner,No,Yes,No,Celluar phone network,1,-1,0,Inexistent,No
3,19300,2008-08-06,39,single,University,Administrator,No,No,No,Celluar phone network,1,-1,0,Inexistent,No
4,34497,2009-05-07,30,married,Middle School,Administrator,No,No,No,Celluar phone network,1,-1,1,Failed,No


In [0]:
# normalization

numeric_features = ['age', 'n_contacts_campaign', 'days_since_last_contact', 'n_contacts_before']
categorical_features = ['marital_status', 'education', 'job', 'credit_default', 'housing_loan', 'personal_loan', 'communication_type', 'previous_conversion']

numeric_transformer = Pipeline([
    ('scaler', MinMaxScaler()),
])
categorical_transformer = Pipeline ([
    ('onehotencoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer([
    ('n_transformer', numeric_transformer, numeric_features),
    ('c_transformer', categorical_transformer, categorical_features),
])

pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('model', model)
])

#pipeline = pipeline.fit(X, y)


In [14]:
n_estimators = randint(200, 400)
max_depth = randint(75, 125)
custom_scorer = make_scorer(f1_score, pos_label='Yes')

# model__ weil man im object "model" in der pipeline die Parameter verbessern/setzen will
param_distributions = {'model__n_estimators': n_estimators, 'model__max_depth': max_depth}

rs = RandomizedSearchCV(pipeline, param_distributions=param_distributions, n_iter=5,
                       scoring=custom_scorer, n_jobs=-1, iid=False, cv=10, random_state=1909)
rs = rs.fit(X, y)

print(f'Optimal parameters: {rs.best_params_}')

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Optimal parameters: {'model__max_depth': 88, 'model__n_estimators': 343}


In [15]:
# best: {'model__max_depth': 96, 'model__n_estimators': 286}
#
hyperparams = {'n_estimators': rs.best_params_['model__n_estimators'], 
               'criterion': 'gini', 
               'max_depth': rs.best_params_['model__max_depth'],
               'min_samples_split': 2,
               'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto',
               'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None,
               'bootstrap': True, 'oob_score': False, 'n_jobs': None, 'random_state': 1909,
               'verbose': 0, 'warm_start': False, 'class_weight': None}

model = RandomForestClassifier(**hyperparams)

pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('model', model)
])

pipeline.fit(X, y)
res_cv = cross_validate(pipeline, X, y, scoring=custom_scorer, cv=10, return_train_score=True)
res_f1_tr = np.mean(res_cv['train_score']) * 100
res_f1_te = np.mean(res_cv['test_score']) * 100
print(f'Average F1 on Training and Test Sets: {res_f1_tr:.2f}%/{res_f1_te:.2f}%')

Average F1 on Training and Test Sets: 82.03%/28.49%
